In [44]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [45]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [46]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [47]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [48]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            self.w1 = np.random.uniform(-3, 3, (9, 9))
            self.w2 = np.random.uniform(-3, 3, (9, 1))
            self.b1 = np.random.uniform(-3, 3, (9, 1))
            self.b2 = np.random.uniform(-3, 3, 1)
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        
        def predict(self, input):
            return (self.w2.T @ sigmoid(self.w1 @ input.T + self.b1) + self.b2).reshape(-1)
        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, 3, (9, 9))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, 3, (9, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, 3, (9, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, 3, 1)

            return w1, w2, b1, b2
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n - n % 2
        
        for i in range(n - n % 2):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")


            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])

    def predict(self, X_test):
        return self.nets[self.best_net].predict(X_test)

In [49]:
regressor = ERegressor(n = 100, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 50)

Epoch 0: MAE: 15.84001299310242
Epoch 1: MAE: 12.160116518184195
Epoch 2: MAE: 9.46738855468636
Epoch 3: MAE: 7.157972534219814
Epoch 4: MAE: 6.078718760070636
Epoch 5: MAE: 5.3154709423402355
Epoch 7: MAE: 5.076937287645865
Epoch 10: MAE: 4.735542393202827
Epoch 11: MAE: 4.486880792228267
Epoch 12: MAE: 3.8995922480462313
Epoch 24: MAE: 3.8323596085522933
Epoch 28: MAE: 3.2412848021623697


In [50]:
y_pred = regressor.predict(scaled_X_test)

In [51]:
y_pred

array([30.08511919, 30.77377575, 19.31188864, 29.62682207, 11.71138863,
       15.44474498, 12.01899416, 29.40205974, 12.92980078, 25.77048839,
       27.94258579, 24.50705998, 29.27802357, 11.86168117, 28.24897748,
       30.72010163, 10.98187081, 30.72660262, 12.01788374, 25.79454759,
       21.34816681, 21.50193462, 27.19592104, 24.52979746, 30.79077001,
       26.3726391 , 18.79797702, 28.46991298, 29.01648896, 27.12497   ,
       11.80916525, 15.32912239, 11.86045187, 24.45383506, 11.95940396,
       30.79631861, 27.75330595, 29.64248341, 23.0065641 , 12.30938232])

In [52]:
mean_absolute_error(y_test, y_pred)

3.026027534455666